In [ ]:
import numpy as np
import os
import cv2

DATADIR = "/content/drive/Shared drives/RAS - LIBRAS/RAS---Libras/dataset/training" # Onde estão sendo armazenados os arquivos
CATEGORIES = ["A","B","C","D","E","F","G","I","L","M","N","O","P","Q","R","S","T","U","V","W","Y"] # Possibilidades de estudo

training_data = []
IMG_SIZE = 64 # Definindo o tamanho da imagem

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR,category) # Caminho com os diretórios das letras
        class_num = CATEGORIES.index(category) # Dando um número para a categoria
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE) #Cada imagem também é convertida na escala de cinza
                new_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE)) # Redimensionando a imagem
                training_data.append([new_array,class_num]) # Inserindo a imagem e sua classificação
                print(category,img)
            except Exception as e: # Erro que pode ocorrer ao importar imagens
                pass

create_training_data()

import random
random.shuffle(training_data) # Embaralhando os dados que a rede vai usar para treinar

X = [] # Features
y = [] # Labels

for features, label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1) # -1: Quantas features tem, IMG_SIZE x IMG_SIZE: tamanho da imagem, 1: indica que o canal de cores é só o cinza (3 se RGB)

import pickle # Salvar os arquivos criados 

pickle_out = open("/content/drive/Shared drives/RAS - LIBRAS/RAS---Libras/output/X_all.pickle","wb") # Salvar X
pickle.dump(X,pickle_out)
pickle_out.close()

pickle_out = open("/content/drive/Shared drives/RAS - LIBRAS/RAS---Libras/output/y_all.pickle","wb") # Salvar y
pickle.dump(y,pickle_out)
pickle_out.close()

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential # Modelo Sequencial
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard # Para visualização do Modelo da Rede Neural
import pickle
import numpy as np

import time

NAME = 'Verificação_de_letras-cnn-64x2-{}'.format(int(time.time()))

tensorboard = TensorBoard(log_dir = 'logs/{}'.format(NAME)) # Para ter uma visualização do treinamento da rede neural

pickle_in = open("/content/drive/Shared drives/RAS - LIBRAS/RAS---Libras/output/X_all.pickle","rb")
X = np.array(pickle.load(pickle_in))

pickle_in = open("/content/drive/Shared drives/RAS - LIBRAS/RAS---Libras/output/y_all.pickle","rb")
y = np.array(pickle.load(pickle_in))

X = np.array(X/255.0) # Simplificando coloração de Pixels para valores entre 0 e 1

model = Sequential([Conv2D(64,(2,2), activation='relu', input_shape = X.shape[1:]), # Primeira camada
                    MaxPooling2D(pool_size=(2,2)),

                    Conv2D(128,(2,2), activation='relu'), # Segunda camada
                    MaxPooling2D(pool_size=(2,2)),

                    Conv2D(256,(2,2), activation='relu'), # Terceira camada
                    MaxPooling2D(pool_size=(2,2)),

                    Conv2D(256,(2,2), activation='relu'), # Quarta camada
                    MaxPooling2D(pool_size=(2,2)),

                    Flatten(),
                    Dropout(.75), # Faz a rede neural não usar todos os neuronios durante o treinamento
                    Dense(512, activation='relu'), # Quinta camada

                    Dense(22, activation='softmax')]) # Camada de Saída

# model.summary()
adam = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer= adam, loss='SparseCategoricalCrossentropy', metrics=['acc'])
model.fit(X, y, batch_size=32, epochs=25, validation_split=0.1, callbacks = [tensorboard])

model.save('/content/drive/Shared drives/RAS - LIBRAS/RAS---Libras/output/treinando_rede_all.model') # Salvando Rede Neural

Epoch 1/25
930/930 [==============================] - 9s 10ms/step - loss: 0.9951 - acc: 0.6796 - val_loss: 0.0833 - val_acc: 0.9755
Epoch 2/25
930/930 [==============================] - 9s 9ms/step - loss: 0.1554 - acc: 0.9451 - val_loss: 0.0415 - val_acc: 0.9879
Epoch 3/25
930/930 [==============================] - 9s 9ms/step - loss: 0.0892 - acc: 0.9690 - val_loss: 0.0135 - val_acc: 0.9967
Epoch 4/25
930/930 [==============================] - 9s 9ms/step - loss: 0.0673 - acc: 0.9770 - val_loss: 0.0080 - val_acc: 0.9982
Epoch 5/25
930/930 [==============================] - 9s 10ms/step - loss: 0.0544 - acc: 0.9822 - val_loss: 0.0089 - val_acc: 0.9973
Epoch 6/25
930/930 [==============================] - 9s 10ms/step - loss: 0.0476 - acc: 0.9838 - val_loss: 0.0065 - val_acc: 0.9979
Epoch 7/25
930/930 [==============================] - 9s 10ms/step - loss: 0.0413 - acc: 0.9870 - val_loss: 0.0039 - val_acc: 0.9985
Epoch 8/25
930/930 [==============================] - 9s 10ms/step - los

In [7]:
import cv2
import tensorflow as tf

CATEGORIES = ["A","B","C","D","E","F","G","I","L","M","N","O","P","Q","R","S","T","U","V","W","Y"] #Possibilidades de estudo


def prepare(filepath):
    IMG_SIZE = 64
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)


model = tf.keras.models.load_model("/content/drive/Shared drives/RAS - LIBRAS/RAS---Libras/output/treinando_rede_all.model")

prediction = model.predict([prepare('/content/drive/Shared drives/RAS - LIBRAS/RAS---Libras/3.png')]) # Quando usar Predict -> Deve ser uma lista
print(prediction)  # will be a list in a list.
valor = np.where(prediction==1)
print(CATEGORIES[int(valor[1])])

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
F
